In [3]:
import numpy as np 
import pandas as pd 
import pickle


In [55]:
dataset_dir = 'davis.txt'
df = pd.read_csv(dataset_dir, header=None, sep=' ')

In [56]:
df.columns  = ['drug_id', 'prot_id', 'drug_smile', 'prot_seq', 'label']
df.head()

,drug_id,prot_id,drug_smile,prot_seq,label
0,11314340,AAK1,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,7.366532
1,11314340,ABL1(E255K),CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000
2,11314340,ABL1(F317I),CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000
3,11314340,ABL1(F317I)p,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000
4,11314340,ABL1(F317L),CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000


In [57]:
from tqdm import tqdm

drug_smiles = {}
prot_seqs= {}

for index, row in tqdm(df.iterrows()):
    drug_id = row['drug_id']
    drug_smile = row['drug_smile']
    prot_id = row['prot_id']
    prot_seq = row['prot_seq']
    
    drug_smiles[drug_id] = drug_smile
    prot_seqs[prot_id] = prot_seq

30056it [00:03, 7807.78it/s]


In [58]:
drug_id_list = df['drug_id'].unique().tolist()
target_id_list = df['prot_id'].unique().tolist()
print(len(drug_id_list))
print(len(target_id_list))

print(len(drug_smiles))
print(len(prot_seqs))


68
442
68
442


# Use Mol2Vec

In [59]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [60]:
pip install mol2vec

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 封装使用Mol2Vec的函数
输入smiles序列，输出分子特征向量+特征矩阵

In [61]:
import numpy as np 
import pandas as pd 
import pickle
from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence
from rdkit import Chem
from tqdm import tqdm

In [62]:
def get_mol2vec(mol2vec_dir, df_dir, db_name, col_names=['drug_id', 'prot_id', 'drug_smile', 'prot_seq', 'label'], embedding_dimension=300, is_debug=False):    
    mol2vec_model = word2vec.Word2Vec.load(mol2vec_dir)
    
    df = pd.read_csv(df_dir, header=None, sep=' ')
    df.columns = col_names
    df.drop_duplicates(subset='drug_id', inplace=True)    
    drug_ids = df['drug_id'].tolist()
    drug_seqs = df['drug_smile'].tolist()
    
    emb_dict = {}
    emb_mat_dict = {}
    length_dict = {}
    
    percent_unknown = []
    bad_mol = 0
    
    # get pretrain feature
    for idx in tqdm(range(len(drug_ids))):
        flag = 0
        mol_miss_words = 0
        
        drug_id = str(drug_ids[idx])
        molecule = Chem.MolFromSmiles(drug_seqs[idx])
        
        try:
            # Get fingerprint from molecule
            sub_structures = mol2alt_sentence(molecule, 2)
        except Exception as e: 
            if is_debug: 
                print (e)
            percent_unknown.append(100)
            continue    
        
        # 存储该分子的子结构特征矩阵
        emb_mat = np.zeros((len(sub_structures), embedding_dimension))
        length_dict[drug_id] = len(sub_structures)
        
        # 遍历分子中每个子结构
        for i, sub in enumerate(sub_structures):
            # Check to see if substructure exists
            try:
                emb_dict[drug_id] = emb_dict.get(drug_id, np.zeros(embedding_dimension)) + mol2vec_model.wv[sub]  
                emb_mat[i] = mol2vec_model.wv[sub]  
            # If not, replace with UNK (unknown)
            except Exception as e:
                if is_debug : 
                    print ("Sub structure not found")
                    print (e)
                emb_dict[drug_id] = emb_dict.get(drug_id, np.zeros(embedding_dimension)) + mol2vec_model.wv['UNK']
                emb_mat[i] = mol2vec_model.wv['UNK']                
                flag = 1
                mol_miss_words = mol_miss_words + 1        
        emb_mat_dict[drug_id] = emb_mat
        
        percent_unknown.append((mol_miss_words / len(sub_structures)) * 100)
        if flag == 1:
            bad_mol = bad_mol + 1
            
    print(f'All Bad Mol: {bad_mol}, Avg Miss Rate: {sum(percent_unknown)/len(percent_unknown)}%')
    
    # 存储pretrain
    dump_data = {
        "dataset": db_name,
        "vec_dict": emb_dict,
        "mat_dict": emb_mat_dict,
        "length_dict": length_dict
    }    
    with open(f'./{db_name}_mol_pretrain.pkl', 'wb+') as f:
        pickle.dump(dump_data, f)    
    


In [63]:
df_dir = 'davis.txt'
mol2vec_dir = 'model_300dim.pkl'
db_name = 'davis'
get_mol2vec(mol2vec_dir, df_dir, db_name)

  0%|          | 0/68 [00:00<?, ?it/s][13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECATION WARNING: please use MorganGenerator
[13:36:53] DEPRECA

All Bad Mol: 68, Avg Miss Rate: 29.937996840244335%


测试dump的结果

In [64]:
with open(f'./davis_mol_pretrain.pkl', 'rb+') as f:
    temp_data = pickle.load(f)    

In [65]:
temp_data.keys()

dict_keys(['dataset', 'vec_dict', 'mat_dict', 'length_dict'])

In [66]:
temp_data['vec_dict']['11314340'].shape

(300,)

In [67]:
temp_data['mat_dict']['11314340'].shape

(79, 300)

## 封装使用ProtVec的函数

In [68]:
def get_protvec(protvec_dir, df_dir, db_name, col_names=['drug_id', 'prot_id', 'drug_smile', 'prot_seq', 'label'], embedding_dimension=100, is_debug=False):        
    protvec_model = pd.read_csv(protvec_dir, delimiter = '\t')
    trigram_dict = {}
    for idx, row in tqdm(protvec_model.iterrows()):
        trigram_dict[row['words']] = protvec_model.iloc[idx, 1:].values.astype(np.float64)
    trigram_list = set(trigram_dict.keys())
    
    df = pd.read_csv(df_dir, header=None, sep=' ')
    df.columns = col_names
    df.drop_duplicates(subset='prot_id', inplace=True)    
    prot_ids = df['prot_id'].tolist()
    prot_seqs = df['prot_seq'].tolist()
    
    emb_dict = {}
    emb_mat_dict = {}
    length_3mer_target = {}


    # get pretrain feature
    for idx in tqdm(range(len(prot_ids))):
        n = 3
        target = prot_seqs[idx]
        prot_id = str(prot_ids[idx])
        split_by_three = [target[i : i + n] for i in range(0, len(target), n)]
        mer_len = len(split_by_three)
        length_3mer_target[prot_id] = mer_len
        
        emb_mat = np.zeros((mer_len, embedding_dimension))
        for i, trigram in enumerate(split_by_three): 
            if len(trigram) == 2: 
                trigram = "X" + trigram
            elif len(trigram) == 1:
                trigram = "XX" + trigram
            if trigram in trigram_list:
                emb_dict[prot_id] = emb_dict.get(prot_id, np.zeros(embedding_dimension))+ trigram_dict[trigram]
                emb_mat[i] = trigram_dict[trigram]
        emb_mat_dict[prot_id] = emb_mat
    # 存储pretrain
    dump_data = {
        "dataset": db_name,
        "vec_dict": emb_dict,
        "mat_dict": emb_mat_dict,
        "length_dict": length_3mer_target
    }    
    with open(f'./{db_name}_prot_pretrain.pkl', 'wb+') as f:
        pickle.dump(dump_data, f)    
    


In [69]:
df_dir = 'davis.txt'
protvec_dir = 'protVec_100d_3grams.csv'
db_name = 'davis'
get_protvec(protvec_dir, df_dir, db_name)

1956it [00:01, 1620.56it/s]

9048it [00:04, 2168.31it/s]
100%|██████████| 442/442 [00:00<00:00, 543.58it/s]


测试

In [70]:
with open(f'./davis_prot_pretrain.pkl', 'rb+') as f:
    temp_data = pickle.load(f)

In [71]:
temp_data.keys()

dict_keys(['dataset', 'vec_dict', 'mat_dict', 'length_dict'])

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

df = pd.read_csv('train.csv')
df.columns = ['drug_id', 'prot_id', 'drug_smile', 'prot_seq', 'label']


In [5]:
# Load embeddings
with open("davis_mol_pretrain.pkl", "rb") as f:
    drug_data = pickle.load(f)
with open("davis_prot_pretrain.pkl", "rb") as f:
    prot_data = pickle.load(f)

drug_vec_dict = drug_data["vec_dict"]
prot_vec_dict = prot_data["vec_dict"]

In [6]:
print(drug_vec_dict)

{'11314340': array([  3.95830963,   2.93815154,  -7.85763519,  17.71204694,
        -3.50420178,  -2.23336343, -21.5599073 ,   5.86845315,
        10.76653852,   6.04943766, -17.13422687,  -9.32985926,
        -2.1896538 ,   7.55290737,  -2.96293235,   1.5602616 ,
         0.76795212,  -3.00399859,  -9.70698863,  17.61283837,
         3.34151972,  15.3801323 ,  19.13005984,  19.06066713,
       -18.60285886, -11.98029935,  -2.28957995,  -6.90183335,
         2.35445693,  -0.71698763,  21.001223  ,  -3.62681363,
        -2.21383775,  -4.55613214,   7.18895411,   0.5648727 ,
         1.16224359,  -5.74806451,  20.60994002,   8.39826511,
         6.54478787,  -7.11433785,  -2.85387617,  -0.80964842,
       -24.04136837,   2.77978708,  -7.01464949,  10.64550184,
        -4.58574378,   3.58586377,  -1.12889408, -15.29230668,
        -4.71421353,  -7.49265424, -11.62828891, -18.17533539,
        -7.16151206,   8.52099189,  -1.27715747,   0.59920235,
        -1.03197937, -14.64294292,   0.768

In [7]:
df['drug_id'] = df['drug_id'].astype(str)

In [8]:
# Filter to only valid entries (both drug and prot embeddings must exist)
valid_rows = df[df['drug_id'].isin(drug_vec_dict) & df['prot_id'].isin(prot_vec_dict)]

# Prepare X and y
x = []
y = []

In [9]:
for _, row in valid_rows.iterrows():
    drug_vec = drug_vec_dict[row['drug_id']]
    prot_vec = prot_vec_dict[row['prot_id']]
    
    combined = np.concatenate([drug_vec, prot_vec])
    x.append(combined)
    y.append(row['label'])

x = np.array(x)
y = np.array(y)

In [10]:
missing_drugs = 0
missing_prots = 0

for _, row in df.iterrows():
    drug_id = row['drug_id']
    prot_id = row['prot_id']

    if drug_id not in drug_vec_dict:
        missing_drugs += 1
    if prot_id not in prot_vec_dict:
        missing_prots += 1

print(f"Missing drug embeddings: {missing_drugs}")
print(f"Missing protein embeddings: {missing_prots}")


Missing drug embeddings: 0
Missing protein embeddings: 0


In [11]:
# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(x, y)

RandomForestRegressor(random_state=42)

In [12]:
df = pd.read_csv('test.csv')
df.columns = ['drug_id', 'prot_id', 'drug_smile', 'prot_seq', 'label']

In [13]:
df['drug_id'] = df['drug_id'].astype(str)

In [16]:
# Filter to only valid entries (both drug and prot embeddings must exist)
valid_rows = df[df['drug_id'].isin(drug_vec_dict) & df['prot_id'].isin(prot_vec_dict)]

# Prepare X and y
x = []
y = []

In [17]:
for _, row in valid_rows.iterrows():
    drug_vec = drug_vec_dict[row['drug_id']]
    prot_vec = prot_vec_dict[row['prot_id']]
    
    combined = np.concatenate([drug_vec, prot_vec])
    x.append(combined)
    y.append(row['label'])

x = np.array(x)
y = np.array(y)

In [18]:
# Evaluate
y_pred = model.predict(x)
mse = mean_squared_error(y, y_pred)
print("Test MSE:", mse)

Test MSE: 0.3285740746492879
